# LSTM-Seq2seq (Sequence to sequence)

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import *
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from utils import train_test_split, X_Y_split_Seq2seq

In [ ]:
data = pd.read_csv('data_droped_nov18_dummy_final.csv')

In [ ]:
data = data.set_index('datetime')
data.index = pd.to_datetime(data.index)
data = data.drop(['Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0', 'diff'],axis=1)
data=data.reindex(columns=['lots_available', 'total_lot',  'carpark_number','x_coord', 'y_coord',
'car_park_decks', 'gantry_height','BASEMENT CAR PARK', 'COVERED CAR PARK','MECHANISED AND SURFACE CAR PARK', 'MULTI-STOREY CAR PARK',
'SURFACE CAR PARK', '7AM-10.30PM', '7AM-7PM', 'NO', 'WHOLE DAY', 'NO.1','SUN & PH FR 1PM-10.30PM', 'SUN & PH FR 7AM-10.30PM', 'NO.2', 
'YES','N', 'Y'])

In [ ]:
data["day_of_week"] = data.index.weekday
data["hour_of_day"] = data.index.hour

In [ ]:
features = ['lots_available','day_of_week','hour_of_day','total_lot', 'carpark_number', 'x_coord', 'y_coord','car_park_decks', 'gantry_height', 'MULTI-STOREY CAR PARK','WHOLE DAY', 
       'NO.1','SUN & PH FR 7AM-10.30PM']

In [ ]:
data = data[features]
data=data.reindex(columns=features)

In [ ]:
data.loc['2016-02-19 11:15:00',:] = np.nan
data.dropna(inplace=True)

In [ ]:
Train, Test = train_test_split(data, test_step_size=673)
train, val = train_test_split(Train, test_step_size=480)

## data normalization using MinMaxScaler, values range from 0 to 1 interval.

In [ ]:
for i in Train.columns:
    scaler = MinMaxScaler()
    
    s_train = scaler.fit_transform(train[i].values.reshape((-1,1)))
    s_val = scaler.transform(val[i].values.reshape((-1,1)))
    s_test = scaler.transform(Test[i].values.reshape((-1,1)))

    s_train = np.reshape(s_train,(len(s_train)))
    s_val = np.reshape(s_val,(len(s_val)))
    s_test = np.reshape(s_test,(len(s_test)))

    train[i] = s_train
    val[i] = s_val
    Test[i] = s_test

In [ ]:
X_train,Y_train= X_Y_split_Seq2seq(train, x_width=16, y_width=4, label_col_no=0)
X_val,Y_val = X_Y_split_Seq2seq(val, x_width=16, y_width=4, label_col_no=0)
X_test,Y_test= X_Y_split_Seq2seq(Test, x_width=16, y_width=4, label_col_no=0)

## model that hyperparameter configuration from literature & keras tuner random search

In [ ]:
def LSTM_seq2seq():
  model = Sequential()
  #encoder
  model.add(InputLayer((X_train.shape[1],X_train.shape[2])))
  model.add(LSTM(100, activation='relu',return_sequences=True))
  model.add(LSTM(100, activation='relu',return_sequences=True))
  model.add(LSTM(100, activation='relu',return_sequences=True))
  model.add(LSTM(100, activation='relu'))
    
  # context vector  
  model.add(RepeatVector(Y_train.shape[1]))

  #decoder
  model.add(LSTM(100, activation='relu',return_sequences=True))
  model.add(LSTM(100, activation='relu',return_sequences=True))
  model.add(LSTM(100, activation='relu',return_sequences=True))
  model.add(LSTM(100, activation='relu',return_sequences=True))


  model.add(TimeDistributed(Dense(1, activation='linear')))
  model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])
  return model

In [ ]:
LSTM_seq2seq_4hr = LSTM_seq2seq()
stop_early = EarlyStopping(monitor='val_loss', patience=3)
history_lstm_seq = LSTM_seq2seq_4hr.fit(X_train,Y_train, validation_data=(X_val,Y_val), epochs=10, callbacks=[stop_early])

In [ ]:
y_pred = LSTM_seq2seq_4hr.predict(X_test)

In [ ]:
y_pred = y_pred.reshape((-1,4))

In [ ]:
mse = mean_squared_error(y_pred=y_pred, y_true=Y_test)
mae = mean_absolute_error(y_pred=y_pred, y_true=Y_test)
rmse = math.sqrt(mean_squared_error(y_pred=y_pred, y_true=Y_test))
r2 = r2_score(y_pred=y_pred, y_true=Y_test)
print(round(mse,5))
print(round(mae,5))
print(round(rmse,5))
print(round(r2,5))

## performance on different time horizon using model with best time window size (4 hour = 16 timestep)

### entire dataset

In [ ]:
LSTM_seq2seq_4hr_whole = LSTM_seq2seq_4hr

In [ ]:
x,y = last_x_y_generator_Seq2seq(val, x_width = 16, y_width = 4, label_col_no=0)

In [ ]:
current_batch = x[:,4:,:]

In [ ]:
#entire dataset
future_len = 40
Test_new = Test[['lots_available', 'carpark_number','day_of_week','hour_of_day']]
Test_new = Test_new.sort_values(by=['carpark_number', 'datetime'])
l=[]
L=pd.DataFrame()
for i in sorted(Test_new.carpark_number.value_counts().keys()):
  inner = Test_new[Test_new.carpark_number == i]
  inner = inner.reset_index()
  inner = inner.iloc[0:future_len,:]
  l.append(inner)
L = L.append(l)
L['value'] = L.index.values
L['seq'] = np.tile(np.repeat(np.array([0,1,2,3,4,5,6,7,8,9]),4),855)
L = L.sort_index()
L = L.sort_values(by=['seq','carpark_number' ])

In [ ]:
future=10
forcast = []
Xin = current_batch
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_whole.predict(Xin, batch_size=855)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*3420:(i*3420)+3420].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*3420:(i*3420)+3420].values.reshape((-1,4,1))

In [ ]:
Y_t = L['lots_available']

l=[]
for i in range(10):
  l.append(Y_t[i*3420:(i*3420)+3420].values.reshape((-1,4)))

In [ ]:
rmse40step=[]
for i in range(10):
  f = forcast[i].reshape(-1,4)
  new_l = np.hsplit(l[i],4)
  new_f = np.hsplit(f,4)#length 4 
  for j in range(4):
    rmse40step.append(math.sqrt(mean_squared_error(y_pred=new_f[j], y_true=new_l[j] )))

In [ ]:
rmse40step = pd.DataFrame(rmse40step)
rmse40step.to_csv('rmse40step855_lstm_seq2seq.csv')

### group level (five regions, 10 to 12 parking lots per region)

In [ ]:
#performance by regions
central = data[(data['x_coord'] >30500) & (data['x_coord'] < 32500)& (data['y_coord'] >35000) & (data['y_coord'] < 36000)]
north_area = data[(data['x_coord'] >25000) & (data['x_coord'] < 26500)& (data['y_coord'] >44000) ]
west_area = data[ (data['x_coord'] < 20000)& (data['y_coord'] >37500) & (data['y_coord'] < 38500)]
east_area = data[ (data['x_coord'] > 35000)& (data['y_coord'] >38000) & (data['y_coord'] < 39000)]
south_area = data[(data['x_coord'] >25000) & (data['x_coord'] < 26000)& (data['y_coord'] >30000) & (data['y_coord'] < 31000)]

In [ ]:
TRAIN_central, TEST_central = train_test_split(central, test_step_size=673)
TRAIN_north, TEST_north = train_test_split(north_area, test_step_size=673)
TRAIN_west, TEST_west = train_test_split(west_area, test_step_size=673)
TRAIN_east, TEST_east = train_test_split(east_area, test_step_size=673)
TRAIN_south, TEST_south = train_test_split(south_area, test_step_size=673)

In [ ]:
train_central,test_central = scaler(TRAIN_central,TEST_central)
train_north, test_north  = scaler(TRAIN_north,TEST_north)
train_west, test_west  = scaler(TRAIN_west,TEST_west)
train_east, test_east  = scaler(TRAIN_east,TEST_east)
train_south, test_south  = scaler(TRAIN_south,TEST_south)

In [ ]:
LSTM_seq2seq_4hr_central = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_north = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_west = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_east = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_south = LSTM_seq2seq_4hr

In [ ]:
x_central, _ = last_x_y_generator_Seq2seq(train_central, x_width = 16, y_width = 4, label_col_no=0)
x_north, _ = last_x_y_generator_Seq2seq(train_north, x_width = 16, y_width = 4, label_col_no=0)
x_west, _ = last_x_y_generator_Seq2seq(train_west, x_width = 16, y_width = 4, label_col_no=0)
x_east, _ = last_x_y_generator_Seq2seq(train_east, x_width = 16, y_width = 4, label_col_no=0)
x_south, _ = last_x_y_generator_Seq2seq(train_south, x_width = 16, y_width = 4, label_col_no=0)

In [ ]:
current_batch_central = x_central[:,4:,:]
current_batch_north = x_north[:,4:,:]
current_batch_west = x_west[:,4:,:]
current_batch_east = x_east[:,4:,:]
current_batch_south = x_south[:,4:,:]

In [ ]:
future_len = 40
Test_new = test_south[['lots_available', 'carpark_number','day_of_week','hour_of_day']]
Test_new = Test_new.sort_values(by=['carpark_number', 'datetime'])
l=[]
L=pd.DataFrame()
for i in sorted(Test_new.carpark_number.value_counts().keys()):
  inner = Test_new[Test_new.carpark_number == i]
  inner = inner.reset_index()
  inner = inner.iloc[0:future_len,:]
  l.append(inner)
L = L.append(l)
L['value'] = L.index.values
L['seq'] = np.tile(np.repeat(np.array([0,1,2,3,4,5,6,7,8,9]),4),10)#need to change based on batch size
L = L.sort_index()
L = L.sort_values(by=['seq','carpark_number' ])

In [ ]:
#central
future=10
forcast = []
Xin = current_batch_central
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_central.predict(Xin, batch_size=11)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*44:(i*44)+44].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*44:(i*44)+44].values.reshape((-1,4,1))

In [ ]:
#north
future=10
forcast = []
Xin = current_batch_north
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_north.predict(Xin, batch_size=11)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*44:(i*44)+44].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*44:(i*44)+44].values.reshape((-1,4,1))

In [ ]:
#west
future=10
forcast = []
Xin = current_batch_west
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_west.predict(Xin, batch_size=12)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*48:(i*48)+48].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*48:(i*48)+48].values.reshape((-1,4,1))

In [ ]:
#east
future=10
forcast = []
Xin = current_batch_east
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_east.predict(Xin, batch_size=11)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*44:(i*44)+44].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*44:(i*44)+44].values.reshape((-1,4,1))

In [ ]:
#south
future=10
forcast = []
Xin = current_batch_south
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_south.predict(Xin, batch_size=10)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*40:(i*40)+40].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*40:(i*40)+40].values.reshape((-1,4,1))

In [ ]:
Y_t = L['lots_available']

l=[]
for i in range(10):
  l.append(Y_t[i*40:(i*40)+40].values.reshape((-1,4)))

In [ ]:
rmse40step=[]
for i in range(10):
  f = forcast[i].reshape(-1,4)
  new_l = np.hsplit(l[i],4)
  new_f = np.hsplit(f,4)#length 4 
  for j in range(4):
    rmse40step.append(math.sqrt(mean_squared_error(y_pred=new_f[j], y_true=new_l[j] )))

In [ ]:
rmse40step = pd.DataFrame(rmse40step)
rmse40step.to_csv('rmse40step_south_lstm_seq2seq.csv')

### individual level (five regions, 1 parking lot per region)

In [ ]:
#individual car park
#central
train_41 =train_central[train_central.carpark_number==0]
test_41 =test_central[test_central.carpark_number==0]
#north
train_547 =train_north[train_north.carpark_number==0]
test_547 =test_north[test_north.carpark_number==0]
#west
train_22 =train_west[train_west.carpark_number==0]
test_22 =test_west[test_west.carpark_number==0]
#east
train_437 =train_east[train_east.carpark_number==0]
test_437 =test_east[test_east.carpark_number==0]
#south
train_514 =train_south[train_south.carpark_number==0]
test_514=test_south[test_south.carpark_number==0]

In [ ]:
LSTM_seq2seq_4hr_41 = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_547 = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_22 = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_437 = LSTM_seq2seq_4hr
LSTM_seq2seq_4hr_514 = LSTM_seq2seq_4hr

In [ ]:
xtrain41 = window_generator_Seq2seq(train_41, x_width=16,y_width=4,label_col_no=0)
xtrain547 = window_generator_Seq2seq(train_547, x_width=16,y_width=4,label_col_no=0)
xtrain22 = window_generator_Seq2seq(train_22, x_width=16,y_width=4,label_col_no=0)
xtrain437 = window_generator_Seq2seq(train_437, x_width=16,y_width=4,label_col_no=0)
xtrain514 = window_generator_Seq2seq(train_514, x_width=16,y_width=4,label_col_no=0)

In [ ]:
last_central = xtrain41[-1:,:,:]
last_north = xtrain547[-1:,:,:]
last_west = xtrain22[-1:,:,:]
last_east = xtrain437[-1:,:,:]
last_south = xtrain514[-1:,:,:]

In [ ]:
current_batch_41 = last_central[:,4:,:]
current_batch_547 = last_north[:,4:,:]
current_batch_22 = last_west[:,4:,:]
current_batch_437 = last_east[:,4:,:]
current_batch_514 = last_south[:,4:,:]

In [ ]:
future_len = 40
Test_new = test_514[['lots_available', 'carpark_number','day_of_week','hour_of_day']]
Test_new = Test_new.sort_values(by=['carpark_number', 'datetime'])
l=[]
L=pd.DataFrame()
for i in sorted(Test_new.carpark_number.value_counts().keys()):
  inner = Test_new[Test_new.carpark_number == i]
  inner = inner.reset_index()
  inner = inner.iloc[0:future_len,:]
  l.append(inner)
L = L.append(l)
L['value'] = L.index.values
L['seq'] = np.tile(np.repeat(np.array([0,1,2,3,4,5,6,7,8,9]),4),1)
L = L.sort_index()
L = L.sort_values(by=['seq','carpark_number' ])

In [ ]:
#central
future=10
forcast = []
Xin = current_batch_41
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_41.predict(Xin, batch_size=1)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*4:(i*4)+4].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*4:(i*4)+4].values.reshape((-1,4,1))

In [ ]:
#north
future=10
forcast = []
Xin = current_batch_547
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_547.predict(Xin, batch_size=1)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*4:(i*4)+4].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*4:(i*4)+4].values.reshape((-1,4,1))

In [ ]:
#west
future=10
forcast = []
Xin = current_batch_22
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_22.predict(Xin, batch_size=1)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*4:(i*4)+4].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*4:(i*4)+4].values.reshape((-1,4,1))

In [ ]:
#east
future=10
forcast = []
Xin = current_batch_437
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_437.predict(Xin, batch_size=1)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*4:(i*4)+4].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*4:(i*4)+4].values.reshape((-1,4,1))

In [ ]:
#south
future=10
forcast = []
Xin = current_batch_514
time=[]
for i in range(future):
    out = LSTM_seq2seq_4hr_514.predict(Xin, batch_size=1)   
    forcast.append(out) 
    print(forcast)
    Xin = insert_end_Seq2seq(Xin,out,16)
    Xin[:,16-4:,1:2] = L[['day_of_week']][i*4:(i*4)+4].values.reshape((-1,4,1))
    Xin[:,16-4:,2:3] = L[['hour_of_day']][i*4:(i*4)+4].values.reshape((-1,4,1))

In [ ]:
Y_t = L['lots_available']

l=[]
for i in range(10):
  l.append(Y_t[i*4:(i*4)+4].values.reshape((-1,4)))

In [ ]:
rmse40step=[]
for i in range(10):
  f = forcast[i].reshape(-1,4)
  new_l = np.hsplit(l[i],4)
  new_f = np.hsplit(f,4)#length 4 
  for j in range(4):
    rmse40step.append(math.sqrt(mean_squared_error(y_pred=new_f[j], y_true=new_l[j] )))

In [ ]:
rmse40step = pd.DataFrame(rmse40step)
rmse40step.to_csv('rmse40step_single_south_lstm_seq2seq.csv')

## robustness check

- based on performance on different time window and different sample size, study determined best time window size is 16 timesteps (4 hour).
- to check the model fit, study tested performance of model on train and test set

In [ ]:
# plot diagnostic learning curves
# plot training and validation loss
plt.figure(figsize= (6,4))
plt.plot()
plt.title('MSE Loss')
plt.plot(history_lstm_seq.history['loss'], color='blue', label='train')
plt.plot(history_lstm_seq.history['val_loss'], color='orange', label='validation')
plt.xlabel('epochs')
plt.ylabel('Score')
plt.legend(['Train', 'Val'])

#plot training and test accuracy	
plt.figure(figsize= (6,4))
plt.plot()
plt.title('RMSE Loss')
plt.plot(history_lstm_seq.history['root_mean_squared_error'], color='blue', label="train")
plt.plot(history_lstm_seq.history['val_root_mean_squared_error'], color='orange', label='validation')
plt.xlabel('epochs')
plt.ylabel('Score')
plt.legend(['Train','Val'])
plt.show()